In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
from matplotlib import pyplot as plt
import scipy.stats as sc
import conf
import importlib
import os
import random
importlib.reload(conf)

<module 'conf' from '/Volumes/homes/Maya/Guests/Amit/CompSagolProj/conf.py'>

In [ ]:
def get_region_mean(region_idx, data, atlas_cifti_data):
    """
    Compute the mean timeseries for a brain region.
    
    Parameters
    ----------
    region_idx : int
        Index of the brain region to extract from the atlas.
    data : np.array
        Functional timeseries data of shape (timepoints, vertices).
    atlas_cifti_data : np.array
        Atlas label data indicating the region assignment for each voxel.

    Returns
    -------
    region_timeseries : np.array
        Mean timeseries of the specified region.
    """
    region_vertices_mask = atlas_cifti_data == region_idx
    region_timeseries = np.mean(data[:,region_vertices_mask[0]], axis = 1)
    return region_timeseries

In [ ]:
def get_same_program_subjs(curr_subj, curr_subj_program, map_prog_to_subj):
    """
    Extract list of subjects with the same academic program affiliation as the current subject given as param

    Parameters
    ----------
    curr_subj : str
        ID of the current subject.
    curr_subj_program : str
        The program that the current subject is enrolled in.
    map_prog_to_subj : dict
        Dictionary mapping each program to a list of subject identifiers.

    Returns
    -------
    same_program_subjs : list of str
        List of subject identifiers in the same program, excluding the current subject.
    """
    return [subj for program, subjs in map_prog_to_subj.items() if program == curr_subj_program for subj in subjs if subj != curr_subj]

In [ ]:
def get_diff_program_subjs(curr_program, map_prog_to_subj, comp_subj_cnt):
    """
    Extract list of subjects with a different academic program affiliation as the current subject given as param, randomly sampling the number of subjects to match the ingroup size for comparison.

    Parameters
    ----------
    curr_program : str
        Name of the current program.
    map_prog_to_subj : dict
        Dictionary mapping each program to a list of subject identifiers.
    comp_subj_cnt : int
        Number of subjects to match the ingroup size for comparison.

    Returns
    -------
    sampled_subjs : list of str
        List of subject identifiers from different programs.
    """
    diff_subj_list =  [subj for program, subjs in map_prog_to_subj.items() if program != curr_program for subj in subjs]
    return random.sample(diff_subj_list, comp_subj_cnt)

In [ ]:
def calc_mean_subjs_data(time_point, category_prefix, relevant_subjs):
    """
    Calculate the mean voxel activity for a group of subjects at a specific time point and stimulus category.

    Parameters
    ----------
    time_point : str
        Identifier for the time point used to locate the fMRI data.
    category_prefix : str
        Prefix specifying the stimulus category, an empty string '' indicates all categories.
    relevant_subjs : list of str
        List of subject identifiers to include in the mean calculation.

    Returns
    -------
    mean_subjs_data : np.array
        average voxel activity across subjects.
    """
    if category_prefix == '': # All categories (6 movies)
        time_series = conf.FMRI_TIMESERIES_ALL_CATEGORIES
    else: # Specific category (2 movies)
        time_series = conf.FMRI_TIMESERIES_ONE_CATEGORY

    # Create an empty matrix to calculate the mean
    mean_subjs_data = np.zeros((time_series, conf.N_VOXELS))

    for subj in relevant_subjs: # Sum relevant subjects data
        
        cifti_path = f'{conf.ROOT_PATH}/{time_point}/{subj}/MNINonLinear/Results/movies_concat_data/{category_prefix}movies_concat_data_AP_Atlas_MSMAll_hp2000_clean.dtseries.nii'
        load_cifti = nib.load(cifti_path)
        cifti_data = load_cifti.get_fdata()
        mean_subjs_data += cifti_data
        
    mean_subjs_data = mean_subjs_data / len(relevant_subjs)
    
    return mean_subjs_data

In [ ]:
# Extract atlas cifti file and number of regions inside it
ATLAS_CIFTI  = nib.load(conf.ATLAS_CIFTI_PATH).get_fdata()
N_REGIONS = int(np.max(ATLAS_CIFTI))

# Load behavioral data of fMRI participants
behav_data_df = pd.read_excel('/Volumes/homes/Maya/Guests/Amit/behav_data.xlsx')
behav_data_df = behav_data_df[~behav_data_df['subj'].astype(str).isin(conf.EXCLUDED)]
behav_data_df['subj_formatted'] = behav_data_df['subj'].astype(str).str.zfill(3)

# Create a dict for mapping program with it's corresponded affiliated subjects
map_prog_to_subj = behav_data_df.groupby('program')['subj_formatted'].apply(list).to_dict()

for time_point in conf.TIME_POINTS:

    for category_prefix in conf.SCANS_CATEGORIES_PREFIX:
        
        # Lists to store data before creating a DataFrame
        same_prog_corr_list = []
        diff_prog_corr_list = []
        
        # Compute mean of diff programs using random sampling to match ingroup size for comparison
        map_prog_to_mean_diff = {}
        for prog in map_prog_to_subj.keys():
            comp_subj_cnt = len(map_prog_to_subj[prog]) - 1
            diff_prog_subjs = get_diff_program_subjs(prog, map_prog_to_subj, comp_subj_cnt)
            print(f"compute mean {prog}")
            map_prog_to_mean_diff[prog] = calc_mean_subjs_data(time_point, category_prefix, diff_prog_subjs)
            
        for idx, row in behav_data_df.iterrows():
            subj = row['subj_formatted']
            
            print(f"Analyzing {time_point} {category_prefix} scan of subject {subj}")
            
            # Create a dict to store current subject data
            same_prog_subj_row = {'subject' : subj}
            diff_prog_subj_row = {'subject' : subj}

            # Load subject's data
            cifti_path = f'{conf.ROOT_PATH}/{time_point}/{subj}/MNINonLinear/Results/movies_concat_data/{category_prefix}movies_concat_data_AP_Atlas_MSMAll_hp2000_clean.dtseries.nii'
            load_cifti = nib.load(cifti_path)
            subj_i_data = load_cifti.get_fdata()

            # Mean of subjects' academic program
            same_prog_subjs = get_same_program_subjs(subj, row['program'], map_prog_to_subj)
            same_program_mean = calc_mean_subjs_data(time_point, category_prefix, same_prog_subjs)
            not_subject_program_mean = map_prog_to_mean_diff[row['program']]

            for region in range(1, N_REGIONS+1):
                # compute region mean from region voxels
                timeseries_subj_i = get_region_mean(region, subj_i_data, ATLAS_CIFTI)
                timeseries_same_program = get_region_mean(region, same_program_mean, ATLAS_CIFTI)
                timeseries_diff_program = get_region_mean(region, not_subject_program_mean, ATLAS_CIFTI)

                # correlation of subject's neural activity in brain region with same program's mean activity
                region_same_program_corr, _ = sc.pearsonr(timeseries_subj_i, timeseries_same_program)
                same_prog_subj_row[str(region)] = np.arctanh(region_same_program_corr) #Fisher's z transformation

                # correlation of subject's neural activity in brain region with different program's mean activity
                region_diff_program_corr, _ = sc.pearsonr(timeseries_subj_i, timeseries_diff_program)
                diff_prog_subj_row[str(region)] = np.arctanh(region_diff_program_corr) #Fisher's z transformation
            
            # Append subject’s data to lists used for df creation.
            same_prog_corr_list.append(same_prog_subj_row)
            diff_prog_corr_list.append(diff_prog_subj_row)

        # Save data to excel
        pd.DataFrame(same_prog_corr_list).to_csv(os.path.join(conf.OUTPUT_PATH_DIR_PREFIX, f'{time_point}_{category_prefix}same_prog_corr.csv'), index=False)
        pd.DataFrame(diff_prog_corr_list).to_csv(os.path.join(conf.OUTPUT_PATH_DIR_PREFIX, f'{time_point}_{category_prefix}diff_prog_corr.csv'), index=False)
                

compute mean Engineering_Biology
compute mean Physics
compute mean Psychology_Biology
compute mean Psychology_Computer_Science
Analyzing first_time_point neutral_ scan of subject 001
Analyzing first_time_point neutral_ scan of subject 002
Analyzing first_time_point neutral_ scan of subject 003
Analyzing first_time_point neutral_ scan of subject 004
Analyzing first_time_point neutral_ scan of subject 005
Analyzing first_time_point neutral_ scan of subject 006
Analyzing first_time_point neutral_ scan of subject 007
Analyzing first_time_point neutral_ scan of subject 008
Analyzing first_time_point neutral_ scan of subject 009
Analyzing first_time_point neutral_ scan of subject 012
Analyzing first_time_point neutral_ scan of subject 014
Analyzing first_time_point neutral_ scan of subject 015
Analyzing first_time_point neutral_ scan of subject 016
Analyzing first_time_point neutral_ scan of subject 018
Analyzing first_time_point neutral_ scan of subject 019
Analyzing first_time_point neutra

In [ ]:
# normalize values within region across subjects

for time_point in conf.TIME_POINTS:

    for category_prefix in conf.SCANS_CATEGORIES_PREFIX:
        # load data
        same_prog_df = pd.read_csv(os.path.join(conf.OUTPUT_PATH_DIR_PREFIX, f'{time_point}_{category_prefix}same_prog_corr.csv'))
        diff_prog_df = pd.read_csv(os.path.join(conf.OUTPUT_PATH_DIR_PREFIX, f'{time_point}_{category_prefix}diff_prog_corr.csv'))

        # concat dfs in order to normalize
        concat_df = pd.concat([same_prog_df, diff_prog_df], ignore_index=True)

        # remove the first column of subject ids to avoid normalizing it
        subjects = concat_df['subject']
        concat_df = concat_df.drop(columns = ['subject'])

        norm_df = (concat_df - concat_df.mean(axis = 0))/concat_df.std(axis = 0)

        # add the subject ids column back
        norm_df.insert(0, 'subject', subjects)

        # divide the combined pdf back to same and diff prog
        norm_same_prog_df = norm_df[:33]
        norm_diff_prog_df = norm_df[-33:]
        
        # save normalized data
        norm_same_prog_df.to_csv(os.path.join(conf.OUTPUT_PATH_DIR_PREFIX, f'normalized_data/norm_{time_point}_{category_prefix}same_prog_corr.csv'), index=False)
        norm_diff_prog_df.to_csv(os.path.join(conf.OUTPUT_PATH_DIR_PREFIX, f'normalized_data/norm_{time_point}_{category_prefix}diff_prog_corr.csv'), index=False)


In [35]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 2.1 MB/s eta 0:00:0000:0100:01
